In [3]:
using Convex
using ECOS
using Clp

x = Variable(4)

# objective
profit = 50*x[1]+40*x[2]+60*x[3]+30*x[4]

# Time constraint
a = [120, 100, 180, 140]
b = 5000

#########################
## NOMINAL FORMULATION ##
#########################

nominal_constrs = [
    x>=0,
    dot(a, x)<=b,
    2*x[1]+3*x[2]+6*x[3]+1*x[4] <= 80,
    3*x[1]+2*x[2]+2*x[3]+0*x[4] <= 40,
]

nominal_problem = maximize(profit, nominal_constrs)
solve!(nominal_problem, ClpSolver())

println("Optimal Nominal Solution: ", x.value)

########################
## ROBUST FORMULATION ##
########################

# Time constraint ellipse
E = diagm(0=>a)*0.3

robust_constrs = [
    x>=0,
    dot(a, x)+norm(sqrt(E)*x)<=b,
    2*x[1]+3*x[2]+6*x[3]+1*x[4] <= 80,
    3*x[1]+2*x[2]+2*x[3]+0*x[4] <= 40,
]

robust_problem = maximize(profit, robust_constrs)
solve!(robust_problem, ECOSSolver(verbose=false))

println("Optimal Robust Solution: ", x.value)

#############################
## LARGE SCALE FORMULATION ##
#############################

function wc_val(a, E, x)
    return a+E*x/sqrt(dot(x, E*x))
end

relaxed_constrs = nominal_constrs
for k in 1:20

    xk=x.value
    a_wc = wc_val(a, E, xk)
    global relaxed_constrs = relaxed_constrs + [dot(a_wc,x)<=b]
    relaxed_problem = maximize(profit, relaxed_constrs)
    solve!(relaxed_problem, ClpSolver())

    println("Relaxed Solution (iteration " * string(k) *") : "* string(x.value)) 
end


Optimal Nominal Solution: [0.6250000000000026; 19.062499999999996; -0.0; 21.5625]
Optimal Robust Solution: [4.237282960867527; 9.831047810920662; 3.8130277477201053; 19.154123954750922]
Relaxed Solution (iteration 1) : [8.285369232583935; -0.0; 7.571946151124092; 17.99758462808757]
Relaxed Solution (iteration 2) : [6.276105660554081; 4.879630709104424; 5.706210800064449; 18.57163175119187]
Relaxed Solution (iteration 3) : [5.274930305693165; 7.3110518898718695; 4.776552651588375; 18.8576678094678]
Relaxed Solution (iteration 4) : [4.762620946865432; 8.555229365986627; 4.300839213715223; 19.004034726017906]
Relaxed Solution (iteration 5) : [4.501842853758574; 9.18854637282429; 4.058689346537842; 19.078539094782922]
Relaxed Solution (iteration 6) : [4.370069871566793; 9.508565856322765; 3.9363293363270415; 19.11618666993586]
Relaxed Solution (iteration 7) : [4.303807528357779; 9.669488380017988; 3.8748003274453398; 19.135117838558433]
Relaxed Solution (iteration 8) : [4.270578563791444; 

In [2]:
robust_problem

maximize
└─ + (affine; real)
   ├─ * (affine; real)
   │  ├─ 50
   │  └─ index (affine; real)
   │     └─ …
   ├─ * (affine; real)
   │  ├─ 40
   │  └─ index (affine; real)
   │     └─ …
   ├─ * (affine; real)
   │  ├─ 60
   │  └─ index (affine; real)
   │     └─ …
   └─ * (affine; real)
      ├─ 30
      └─ index (affine; real)
         └─ …
subject to
├─ >= constraint (affine)
│  ├─ 4-element real variable (id: 143…758)
│  └─ 0
├─ <= constraint (convex)
│  ├─ + (convex; real)
│  │  ├─ sum (affine; real)
│  │  │  └─ …
│  │  └─ norm2 (convex; positive)
│  │     └─ …
│  └─ 5000
├─ <= constraint (affine)
│  ├─ + (affine; real)
│  │  ├─ * (affine; real)
│  │  │  ├─ …
│  │  │  └─ …
│  │  ├─ * (affine; real)
│  │  │  ├─ …
│  │  │  └─ …
│  │  ├─ * (affine; real)
│  │  │  ├─ …
│  │  │  └─ …
│  │  └─ * (affine; real)
│  │     ├─ …
│  │     └─ …
│  └─ 80
└─ <= constraint (affine)
   ├─ + (affine; real)
   │  ├─ * (affine; real)
   │  │  ├─ …
   │  │  └─ …
   │  ├─ * (affine; real)
   │  │  ├─ …